In [4]:
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI\\research'

In [5]:
import os
os.chdir("../")
%pwd

'c:\\Users\\akish\\SentimentAnalysis\\SentimentAPI'

In [32]:
import pandas as pd
import tensorflow as tf
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch
pd.set_option('display.max_colwidth', None)
MODEL_NAME = 'distilbert-base-uncased-finetuned-sst-2-english'
BATCH_SIZE = 16
N_EPOCHS = 3

In [38]:
sentiData = pd.read_excel('artifacts\data_transformation\sentimentDataset\TransformedFinalData.xlsx')

In [34]:
class CustomDataset:
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer(text, truncation=True, padding="max_length",
                              max_length=self.max_len)

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label
    }

In [45]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased'
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AssertionError: Torch not compiled with CUDA enabled

In [47]:
%pip install cuda-python


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
    --------------------------------------- 0.2/10.2 MB 6.7 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/10.2 MB 7.0 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.2 MB 7.9 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/10.2 MB 7.3 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/10.2 MB 7.3 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/10.2 MB 7.3 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/10.2 MB 3.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/10.2 MB 4.1 MB/s eta 0:00:03
   ------- -------------------------------- 1.9/10.2 MB 4.7 MB/s eta 0:00:02
   -------- ------------------------------- 2.2/10.2 MB 4.8 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.2 MB 5.2 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.2 MB 5.4 MB/s eta 0:00:02
   --

In [49]:
%pip install torch==1.4.0+cu92 torchvision==0.5.0+cu92 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.htmlNote: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.25.0 requires torch>=1.10.0, but you have torch 1.4.0+cu92 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 0.0/641.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/641.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/641.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/641.9 MB ? eta -:--:--
     ---------------------------------------- 0.0/641.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/641.9 MB 2.6 MB/s eta 0:04:11
     ---------------------------------------- 0.1/641.9 MB 2.6 MB/s eta 0:04:11
     ---------------------------------------- 0.1/641.9 MB 2.6 MB/s eta 0:04:11
     ---------------------------------------- 0.1/641.9 MB 2.6 MB/s eta 0:04:11
     -------------------------------------- 0.1/641.9 MB 554.9 kB/s eta 0:19:17
     -------------------------------------- 0.1/641.9 MB 554.9 kB/s eta 0:19:17
     -------------------------------------- 0.1/641.9 MB 554.9 kB/s eta 0:19:17
     -------------------------------------- 0.2/641.9 MB 436.9 kB/s eta 0:

In [53]:
import torch
torch.cuda.is_available()

False

In [39]:
X = sentiData['Review'].tolist()

y = sentiData['sentiment'].tolist()

dataset = CustomDataset(X, y, tokenizer)

In [40]:
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

In [41]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(example):
  labels = example.label_ids
  preds = example.predictions.argmax(-1)

  f1 = f1_score(labels, preds, average="weighted")
  acc = accuracy_score(labels, preds)

  return {'accuracy': acc, "f1": f1}

In [43]:
%pip install accelerate -U


   ---------------------------------------- 0.0/265.7 kB ? eta -:--:--
   ---------------- ----------------------- 112.6/265.7 kB 3.3 MB/s eta 0:00:01
   ---------------------------------------- 265.7/265.7 kB 3.3 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from transformers import Trainer, TrainingArguments
batch_size = 16
model_name = "distilbert_finetuned_setiment"

args = TrainingArguments(
    output_dir = "output",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size = batch_size,
    learning_rate = 2e-5,
    num_train_epochs = 1,
    evaluation_strategy = 'epoch'
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [10]:
# #define a tokenizer object
# tokenizer = DistilBertTokenizer.from_pretrained(MODEL_NAME)
# #tokenize the text
# train_encodings = tokenizer(list(trainSentences.values),
#                             truncation=True, 
#                             padding=True)
# test_encodings = tokenizer(list(testSentences.values),
#                            truncation=True, 
#                            padding=True)

ValueError: Input is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [52]:
%pip install nvidia-cuda-runtime-cu12

   ---------------------------------------- 0.0/862.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/862.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/862.5 kB ? eta -:--:--
   -- ------------------------------------- 61.4/862.5 kB 1.7 MB/s eta 0:00:01
   --------- ------------------------------ 194.6/862.5 kB 2.4 MB/s eta 0:00:01
   ----------- ---------------------------- 245.8/862.5 kB 2.2 MB/s eta 0:00:01
   ----------- ---------------------------- 256.0/862.5 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 256.0/862.5 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 256.0/862.5 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 256.0/862.5 kB 1.4 MB/s eta 0:00:01
   ----------- ---------------------------- 256.0/862.5 kB 1.4 MB/s eta 0:00:01
   ------------ ------------------------- 276.5/862.5 kB 630.9 kB/s eta 0:00:01
   ------------ ------------------------- 286.7/862.5 kB 610.3 kB/s 


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
!nvidia-smi

Sat Jan  6 21:41:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.97                 Driver Version: 531.97       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1650       WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8                2W /  N/A|     27MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--